**Import modules et données puis préparation**

In [1]:
import math
from random import randrange, randint
import numpy as np
import pandas as pd
df = pd.read_csv("info_communes_parc.csv", sep=";")

In [2]:
elite = 0.2
tx_mutations = 0.1
taille_echantillon = 5000
trajets = pd.DataFrame({"itineraire":[],"kms":[],"pois":[],"hotel_j3":[],"hotel_j7":[]})
nb_etapes = 10

In [3]:
def distance (depart, arrivee) :
    tmp = df.loc[df["insee"].isin([depart,arrivee]), "lat_centre":"long_centre"]
    x_dep = tmp.iloc[0,1]
    y_dep = tmp.iloc[0,0]
    x_arr = tmp.iloc[1,1]
    y_arr = tmp.iloc[1,0]
    distance = math.acos(math.sin(math.radians(x_dep))*math.sin(math.radians(x_arr))+math.cos(math.radians(x_dep))*math.cos(math.radians(x_arr))*math.cos(math.radians(y_dep-y_arr)))*6371
    return round(distance,1)
noms_villes = df["insee"].tolist()

In [4]:
def generateur_aleatoire (n_etapes, n_iterations, liste) :
    trajets = []
    for h in range(n_iterations) :
        liste_voyage = []
        tmp_liste = liste.copy()
        for i in range(n_etapes) :
            nombre_alea = randrange(0, len(tmp_liste), 1)
            liste_voyage.append(tmp_liste[nombre_alea])
            tmp_liste.pop(nombre_alea)
        trajets.append(liste_voyage)
    return trajets

In [5]:
trajets["itineraire"] = generateur_aleatoire(nb_etapes,taille_echantillon, noms_villes)

In [7]:
def notation () :
    #pense bete tri colonnes array[array[:, 1].argsort()]
    global trajets
    #decompte distance
    for i in range(taille_echantillon) :
        distance_totale = 0
        nb_poi = 0
        print("ligne "+ str(i),end='\r')
        for j in range(9) :
            distance_totale += distance(trajets.iloc[i,0][j],trajets.iloc[i,0][j+1])
        trajets.iloc[i,1] = distance_totale
    #decompte POI
        for k in range(10) :
            valeur = df.loc[df["insee"] == trajets.iloc[i,0][k], "nb_poi"].iloc[0]
            nb_poi += valeur
        trajets.iloc[i,2] = nb_poi
        trajets.iloc[i,3] = df.loc[df["insee"] == trajets.iloc[i,0][2], "nb_hotels_3_etoiles_min"].iloc[0]
        trajets.iloc[i,4] = df.loc[df["insee"] == trajets.iloc[i,0][6], "nb_hotels_3_etoiles_min"].iloc[0]

In [8]:
notation()

In [9]:
trajets = trajets.sort_values(by="kms",ascending=True)
trajets["decile_km"] = 0
for i in range(trajets.shape[0]) :
    trajets.iloc[i,5] = math.floor(i / (taille_echantillon / 10)) + 1
trajets = trajets.sort_values(by="pois",ascending=False)
trajets["decile_poi"] = 0
for i in range(trajets.shape[0]) :
    trajets.iloc[i,6] = math.floor(i / (taille_echantillon / 10)) + 1
trajets["decile_cumul"] = 10
for i in range(trajets.shape[0]) :
    for j in range(1,10) :
        if trajets.iloc[i,5] == j and trajets.iloc[i,6] == j :
            trajets.iloc[i,7] = j
trajets = trajets.sort_values(by=["decile_cumul","pois","kms"],ascending=[True,False,True])

In [10]:
if trajets[trajets["decile_cumul"]!=10].shape[0] < math.floor(taille_echantillon * elite) :
    nextgen = trajets.iloc[0:trajets[trajets["decile_cumul"]!=10].shape[0],0].tolist()
else :
    nextgen = trajets.iloc[0:math.floor(taille_echantillon * elite),0].tolist()

In [10]:
def mutations (tx_mutation) :
    global trajets
    tmp_liste = trajets.sample(frac=tx_mutation).iloc[:,0].tolist()
    liste_mutants = []
    for genome in tmp_liste :
        alea = randint(0,2)
        if alea == 0 :
            gene1 = randrange(0,10)
            gene2 = gene1
            while gene2 == gene1 :
                gene2 = randrange(0,10)
            genome[gene1], genome[gene2] = genome[gene2], genome[gene1]
            liste_mutants.append(genome)
        elif alea == 1 :
            gene1 = randrange(0,10)
            gene2 = gene1
            while gene2 == gene1 :
                gene2 = randrange(0,10)
            gene_mutation = genome[gene2]
            del(genome[gene2])
            genome.insert(gene1 + 1, gene_mutation)
            liste_mutants.append(genome)
        else :
            debut = randrange(0,10)
            fin = randrange(debut + 1,10) if debut != 9 else 9
            debut_liste = genome[0:debut]
            liste_inverse = genome[debut:fin + 1]
            liste_inverse.reverse()
            fin_liste = genome[fin + 1 : 10]
            genome = debut_liste + liste_inverse + fin_liste
            liste_mutants.append(genome)
    return liste_mutants

In [11]:
lesmutants = mutations(tx_mutations)

In [12]:
nb_croisements = taille_echantillon - len(nextgen) - len(lesmutants)
def croisements(nb_croisements) :
    global trajets
    tmp_liste = trajets.iloc[:,0].tolist()
    liste_croisements = []
    for i in range(nb_croisements) :
        genome_enfant = []
        parent1 = randrange(0,len(tmp_liste))
        parent2 = parent1
        while parent2 == parent1 :
            parent2 = randrange(0,len(tmp_liste))
        for j in range(10) :
            if tmp_liste[parent1][j] not in genome_enfant and tmp_liste[parent2][j] not in genome_enfant :
                alea = randint(0,1)
                genome_enfant.append(tmp_liste[parent1][j]) if alea == 0 else genome_enfant.append(tmp_liste[parent2][j])
            elif (tmp_liste[parent1][j] in genome_enfant and tmp_liste[parent2][j] not in genome_enfant) or (tmp_liste[parent1][j] not in genome_enfant and tmp_liste[parent2][j] in genome_enfant):
                genome_enfant.append(tmp_liste[parent2][j]) if tmp_liste[parent1][j] in genome_enfant else genome_enfant.append(tmp_liste[parent1][j])
            else :
                genes_disponibles = []
                for k in tmp_liste[parent1] + tmp_liste[parent2] :
                    if k not in genes_disponibles and k not in genome_enfant :
                        genes_disponibles.append(k)
                genome_enfant.append(genes_disponibles[randrange(0,len(genes_disponibles))])
        liste_croisements.append(genome_enfant)
    return liste_croisements

In [18]:
lesenfants = croisements(nb_croisements)

In [35]:
prompt = ""
for i in trajets[(trajets["hotel_j3"] > 0) & (trajets["hotel_j7"] > 0)].iloc[0,0] :
    prompt += df.loc[df["insee"] == i, "nom"].iloc[0] + " "
prompt += " - " + str(round(trajets[(trajets["hotel_j3"]> 0) & (trajets["hotel_j7"]> 0)].iloc[0,1],1)) + " kms"
prompt += " - " + str(trajets[(trajets["hotel_j3"]> 0) & (trajets["hotel_j7"]> 0)].iloc[0,2]) + " pois"
print (prompt)

Liernais Villiers-en-Morvan Champeau-en-Morvan Châtin La Celle-en-Morvan Glux-en-Glenne Montlay-en-Auxois Vauclaix Avallon Bazoches  - 266.2 kms - 56.0 pois
